In [1]:
# data processing
import pandas as pd 
from pandas import Series,DataFrame

# linear algebra
import numpy as np 

# data visualization
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns


# Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import re
from sklearn import linear_model
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC

In [ ]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [ ]:
train_df = train.drop(['Ticket'], axis=1)
test_df = test.drop(['Ticket'], axis=1)

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

In [ ]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

In [ ]:
train_df[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean()

In [ ]:
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean()

In [ ]:
g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
grid = sns.FacetGrid(train, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

In [ ]:
g = sns.heatmap(train[["Survived","SibSp","Parch","Age","Fare"]].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")


In [ ]:
train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
test_df['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

In [ ]:
pd.crosstab(test_df['Title'], test_df['Sex'])

In [ ]:

test_df['Title'] = test_df['Title'].replace(['Col','Dona', 'Dr', 'Rev'], 'Rare')

test_df['Title'] = test_df['Title'].replace('Ms', 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
train_df['Title'] = train_df['Title'].fillna(0)

In [ ]:
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
test_df['Title'] = test_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].fillna(0)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df['Sex'] = train_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
test_df['Sex'] = test_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [ ]:
train_df.head()

In [ ]:
train_df.groupby('Title')['Age'].mean()

In [ ]:
test_df.groupby('Title')['Age'].mean()

In [ ]:
## Assigning the NaN Values with the Ceil values of the mean ages
train_df.loc[(train_df.Age.isnull())&(train_df.Title=='Mr'),'Age']=33
train_df.loc[(train_df.Age.isnull())&(train_df.Title=='Mrs'),'Age']=36
train_df.loc[(train_df.Age.isnull())&(train_df.Title=='Master'),'Age']=5
train_df.loc[(train_df.Age.isnull())&(train_df.Title=='Miss'),'Age']=22
train_df.loc[(train_df.Age.isnull())&(train_df.Title=='Other'),'Age']=46

In [ ]:
## Assigning the NaN Values with the Ceil values of the mean ages
test_df.loc[(test_df.Age.isnull())&(test_df.Title=='Mr'),'Age']=32
test_df.loc[(test_df.Age.isnull())&(test_df.Title=='Mrs'),'Age']=39
test_df.loc[(test_df.Age.isnull())&(test_df.Title=='Master'),'Age']=8
test_df.loc[(test_df.Age.isnull())&(test_df.Title=='Miss'),'Age']=22
test_df.loc[(test_df.Age.isnull())&(test_df.Title=='Other'),'Age']=44

In [ ]:
train_df.Age = train_df.Age.replace(np.nan)
train_df["Age"].dropna(how ='any', inplace = True)

In [ ]:
test_df.Age = test_df.Age.replace(np.nan)
test_df["Age"].dropna(how ='any', inplace = True)

In [ ]:
train_df.Age.isnull().any()

In [ ]:
test_df.Age.isnull().any()

In [ ]:
train_df["Age"] = train_df["Age"].astype(int)

In [ ]:
test_df["Age"] = test_df["Age"].astype(int)

In [ ]:
train_df['age']=0
train_df.loc[ train_df['Age'] <= 16, 'age'] = 0
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 32), 'age'] = 1
train_df.loc[(train_df['Age'] > 32) & (train_df['Age'] <= 48), 'age'] = 2
train_df.loc[(train_df['Age'] > 48) & (train_df['Age'] <= 64), 'age'] = 3
train_df.loc[ train_df['Age'] > 64, 'age'] = 4

In [ ]:
test_df['age']=0
test_df.loc[ test_df['Age'] <= 16, 'age'] = 0
test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 32), 'age'] = 1
test_df.loc[(test_df['Age'] > 32) & (test_df['Age'] <= 48), 'age'] = 2
test_df.loc[(test_df['Age'] > 48) & (test_df['Age'] <= 64), 'age'] = 3
test_df.loc[ test_df['Age'] > 64, 'age'] = 4

In [ ]:
train_df

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)

In [ ]:

test_df['Embarked'] = test_df['Embarked'].fillna(freq_port)

In [ ]:
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

In [ ]:
train_df['Embarked'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_df['Embarked'] = test_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
test_df.isnull().values.any().sum()

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [ ]:
train_df[['age', 'Survived']].groupby(['age'], as_index=False).mean()

In [ ]:
train_df.isnull().values.any().sum()

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean()


In [ ]:
test_df['FareBand'] = pd.qcut(test_df['Fare'], 4)

In [ ]:
train_df.loc[ train_df['Fare'] <= 7.91, 'Fare'] = 0
train_df.loc[(train_df['Fare'] > 7.91) & (train_df['Fare'] <= 14.454), 'Fare'] = 1
train_df.loc[(train_df['Fare'] > 14.454) & (train_df['Fare'] <= 31), 'Fare']   = 2
train_df.loc[ train_df['Fare'] > 31, 'Fare'] = 3
train_df['Fare'] = train_df['Fare'].astype(int)

In [ ]:
train_df[['Fare', 'Survived']].groupby(['Fare'], as_index=False).mean()

In [ ]:
test_df.loc[ test_df['Fare'] <= 7.91, 'Fare'] = 0
test_df.loc[(test_df['Fare'] > 7.91) & (test_df['Fare'] <= 14.454), 'Fare'] = 1
test_df.loc[(test_df['Fare'] > 14.454) & (test_df['Fare'] <= 31), 'Fare']   = 2
test_df.loc[ test_df['Fare'] > 31, 'Fare'] = 3
test_df['Fare'] = test_df['Fare'].astype(int)

In [ ]:
train_df = train_df.drop(['FareBand'], axis=1)
test_df = test_df.drop(['FareBand','PassengerId'], axis=1)

In [ ]:
train_df

In [ ]:
train_df.Cabin.unique()

In [ ]:
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

train_df['Cabin'] = train_df['Cabin'].fillna("U0")
train_df['Deck'] = train_df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
train_df['Deck'] = train_df['Deck'].map(deck)
train_df['Deck'] = train_df['Deck'].fillna(0)
train_df['Deck'] = train_df['Deck'].astype(int)

In [ ]:
test_df['Cabin'] = test_df['Cabin'].fillna("U0")
test_df['Deck'] = test_df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
test_df['Deck'] = test_df['Deck'].map(deck)
test_df['Deck'] = test_df['Deck'].fillna(0)
test_df['Deck'] = test_df['Deck'].astype(int)

In [ ]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [ ]:
train_df = train_df.drop(['Age'], axis=1)
test_df = test_df.drop(['Age'], axis=1)

In [ ]:
train_df.head()

In [ ]:
g = sns.heatmap(train_df[["Survived","Pclass","age","Sex","Fare","Title","SibSp","Parch","Embarked",'Deck']].corr(),cmap="BrBG",annot=True)

In [ ]:
test_df.head()

In [ ]:
g = sns.heatmap(test_df[["Pclass","age","Sex","Fare","Title","SibSp","Parch","Embarked",'Deck']].corr(),cmap="BrBG",annot=True)


In [ ]:
trainf = train_df.copy()
testf = test_df.copy()

In [ ]:
X_train = trainf.drop("Survived", axis=1)
Y_train = trainf["Survived"]
X_test  = testf.copy()

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

In [ ]:
acc_decision_tree